In [145]:
import numpy as np
import pandas as pd
import re

In [146]:
df_prod = pd.read_csv('Техрежим доб.CSV', delimiter=';', encoding='utf-8', low_memory=False)

### Приводим дату в конвертируемый формат во фрейме с тех.режимом добывающих скважин

In [147]:
df_prod['Дата'] = df_prod['Дата'].str.split('.')
df_prod['Дата'].map(lambda x: x.insert(0, '01'))

def convert_month_to_digit_mode(date_as_list):
    result = date_as_list
    dict_of_date = {'янв': '1', 'фев': '2', 'мар': '3', 'апр': '4', 'май': '5', 'июн': '6', 'июл': '7', 'авг': '8', 'сен': '9', 'окт': '10', 'ноя': '11', 'дек': '12'}
    result[1] = dict_of_date[result[1]]
    result[0], result[1] = result[1], result[0]


df_prod['Дата'].apply(convert_month_to_digit_mode)
df_prod['Дата'] = df_prod['Дата'].map(lambda x: '/'.join(x))
df_prod['Дата'] = df_prod['Дата'].apply(lambda x: pd.to_datetime(x))

### Подгружаем МЭР и работаем с колонкой Дата, для приведения её к одинаковому формату с колонкой Дата в тех.режиме.

In [148]:
df_mer = pd.read_csv('МЭР.CSV', delimiter=';', low_memory=False)

### Последняя строка в МЭР итоговая по всем строкам, удалим ёё за ненадобностью

In [149]:
df_mer = df_mer.drop(index=df_mer.index[-1])

In [150]:
mer_date = df_mer['Дата'].astype(str)
mer_date = mer_date.str.split('.')
mer_date.map(lambda x: x.insert(0, '01'))


def convert_month_to_digit_mer(date_as_list):
    result = date_as_list
    result[0], result[1] = result[1], result[0]


mer_date.apply(convert_month_to_digit_mer)
mer_date = mer_date.map(lambda x: '/'.join(x))
mer_date = pd.to_datetime(mer_date, errors='coerce')

df_mer.loc[:, 'Дата'] = mer_date

### В МЭР жидкость дана в тоннах и накопленная за месяц в м3, в тех. режим необходимо внести данные по добыче жидкости в м3.
### В таблице МЭР колонку Жидкость м3 необходимо привести к формату числа с плавающей точкой

In [151]:
df_mer['Жидкость, м3'] = df_mer['Жидкость, м3'].str.replace(',', '.')

In [152]:
df_mer['Жидкость, м3'] = df_mer['Жидкость, м3'].astype(float)

### В тех. режиме добывающих скважин колонка со скважинами именуется, как Номер скважины, а в МЭР, как Скважина.
### Приведем все к одному виду для возможности слияния.

In [153]:
df_prod = df_prod.rename(columns={'Номер скважины': 'Скважина'})

In [154]:
df_mer_for_merge = df_mer[['Скважина', 'Дата', 'Время работы, ч', 'Qн, т/сут', 'Жидкость, м3']]

### Делаем слияние по пересекающимся значениям колонок "Скважина" и "Дата"

In [155]:
df_prod = df_prod.merge(df_mer_for_merge, how='inner', on=['Скважина', 'Дата'])

### Заполняем пропущенные значения в колонках "Qж, м3/сут", "Qн, т/сут", "Время работы, ч"

In [156]:
df_prod['Время работы, ч_x'] = df_prod['Время работы, ч_x'].fillna(df_prod['Время работы, ч_y'])
df_prod['Qн, т/сут_x'] = df_prod['Qн, т/сут_x'].fillna(df_prod['Qн, т/сут_y'])
# df_prod['Qж, м3/сут'] = df_prod['Qж, м3/сут'].fillna(df_prod['Жидкость, м3'] / (df_prod['Время работы, ч_x'] / 24))

### Приводим выходной файл тех.режима по добыче в итоговый для загрузки вид

In [157]:
list_of_columns = ['Скважина', 'Куст', 'Дата', 'Состояние на конец месяца', 'Способ эксплуатации', 'Рентабельность', 'Внутренний диаметр эксплуатационной колоны, мм', 'Пласт', 'Qн, т/сут_x', 'Qж, м3/сут',
                   'Обводненность (объемная), %', 'Рзаб, атм', 'Pпл, атм', 'Коэф. продуктивности, м3/сут/атм', 'KH, мД м', 'Скин-фактор', 'Радиус контура питания, м', 'Динамический уровень, м', 'Буферное давление, атм',
                   'Pлин, атм', 'Pзатр, атм', 'Давление на приеме насоса, атм', 'Статический уровень', 'Рзатр при Нстат, атм', 'Тип насоса', 'Дата изм. параметров насоса', 'Глубина спуска насоса, м',
                   'Номинальный напор ЭЦН, м', 'Частота работы ЭЦН, Гц', 'Сила тока ЭЦН, А', 'Номинальная производительность, м3/сут', 'Тип пакера', 'Дата установки пакера', 'Глубина установки пакера, м',
                   'Диаметр штуцера, мм', 'В-сть нефти в пластовых условиях, сПз', 'Плотность нефти (агента закачки для нагнетательных) в поверхностных условиях', 'Объемный коэффициент нефти, м3/м3',
                   'Замеренный газовый фактор, м3/т', 'Глубина верхних дыр перфорации, м', 'Удлинение, м', 'Перфорированная мощность, м', 'Нефтенасыщенная (для добывающих) / эффективная (для нагнетательных) толщина, м',
                   'Внешний диаметр НКТ, мм', 'Вязкость жидкости (агента закачки для нагнетательных) в поверхностных условиях, сПз', 'Добыча нефти, т', 'Добыча жидкости/закачка агента для нагнетательных, м3',
                   'Время работы, ч_x', 'Время работы в периодическом режиме / под циклической закачкой, ч', 'Дебит нефти потенциальный (технологический), т/сут', 'Дебит жидкости потенциальный (технологический), м3/сут',
                   'Плотность воды в пластовых условиях, г/см3', 'Qж с поправкой на диаметр эксп. колонны (технологический), м3/сут', 'Пуск', 'Остановка', 'Проницаемость', 'Тип', 'Примечание']

In [158]:
df_prod = df_prod[list_of_columns]
df_prod = df_prod.rename(columns={'Скважина': 'Номер скважины','Qн, т/сут_x': 'Qн, т/сут', 'Время работы, ч_x': 'Время работы, ч'})

### Заполняем пропущенные значения для тех. режима нагнетательных скважин.

In [159]:
df_inj = pd.read_csv('Техрежим наг.CSV', delimiter=';', encoding='utf-8', low_memory=False)

### Приводим дату в конвертируемый формат во фрейме с тех.режимом нагнетательных скважин

In [160]:
df_inj['Дата'] = df_inj['Дата'].str.split('.')
df_inj['Дата'].map(lambda x: x.insert(0, '01'))

df_inj['Дата'].apply(convert_month_to_digit_mode)
df_inj['Дата'] = df_inj['Дата'].map(lambda x: '/'.join(x))
df_inj['Дата'] = df_inj['Дата'].apply(lambda x: pd.to_datetime(x))

### Переименовываем объекты разработки

In [161]:
list_of_objects_of_field = {'АВ1/3': 'АВ1_3', 'ЮВ1/1': 'ЮВ1_1'}

In [162]:
df_prod['Пласт'] = df_prod['Пласт'].replace(list_of_objects_of_field)
df_inj['Пласт'] = df_inj['Пласт'].replace(list_of_objects_of_field)

In [163]:
df_prod.to_excel('Техрежим доб.xlsx', index=False)

In [164]:
df_inj.to_excel('Техрежим наг.xlsx', index=False)